<a href="https://colab.research.google.com/github/danieloselu3/phase4-project/blob/main/Tweets_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tweets Sentiment Analysis** 

**Business Understanding**

In a world where technology startups are common, consumer perception of a brand can provide us with valuable information about their purchasing behavior and, in turn, the financial performance of the business that produces them. Safaricom, a technology investment firm, has hired us to help them develop a predictive model that keeps track of recent tweets about tech products so they can make smart investment choices. In order to determine which brands to research further for potential investment, Safaricom wants a generalizable model to measure sentiment across various brands. Because people only purchase items they like, Safaricom is only interested in whether consumers feel positively about the brand.

**Data Understanding**

Data.world, through CrowdFlower, provided the dataset. Over 9,000 Tweets were rated as either positive, negative, or neither by human raters. The tweets are from the South by South West conference, and the majority of them are about Apple and Google products. The information was assembled in 2013.
Tweets are typically brief and emotional. They do a great job of capturing consumers' reactions to brands in real time. South by Southwest features the most recent technology and enables customers to contrast products from leading tech companies side by side, which can somewhat lessen their biases.
The tweets with positive sentiment and the tweets without positive sentiment, which include neutral, negative, and indistinguishable sentiments, are the two categories into which we engineered our target.Only the sentiment of the tweet matters to us because it can increase sales and therefore return on investment.

